In [2]:
from app.uzt_client import UZTClient

In [3]:
import main

In [4]:
resp = main.get_categories_from_api()

In [6]:
resp

FResp(status_ok=False, data='No data')

In [7]:
import re
from pydantic import BaseModel
import json

In [8]:
class Job(BaseModel):
    date1: str
    date2: str
    title: str
    company: str
    place: str


In [9]:
class Category(BaseModel):
    name: str
    href: str

In [10]:
#getting categories list
def get_categories_from_page(client):
    categories = list()
    a_list = client.tree.css("#ctl00_MainArea_UpdatePanel1 li a")
    for a in a_list:
        category = Category(
            name = a.text(),
            href = a.attrs.get('href')
        )
        categories.append(category)
    
    return categories
    

In [11]:
#getting href to next page
def get_next_page_href(client):
    a = client.tree.css_first("table.Pager td:last-child a")
    if not a:
        return False
    return a.attrs.get("href")


In [12]:
uzt = UZTClient()
uzt.get(url="https://portal.uzt.lt/LDBPortal/Pages/ServicesForEmployees.aspx")

/mnt/1TB/Coding/Parsing/Projects/parsing_uzt_2/webscraper


<Response [200 OK]>

In [13]:
cats = main.get_categories_from_page(uzt)

In [14]:
cats

FResp(status_ok=True, data=[{'name': 'Administravimas, ekonomika', 'href': "javascript:__doPostBack('ctl00$MainArea$GroupedPOSearchTab$ProffessionGroups$ctl00$ProfGroup','')"}, {'name': 'Amatininkai', 'href': "javascript:__doPostBack('ctl00$MainArea$GroupedPOSearchTab$ProffessionGroups$ctl01$ProfGroup','')"}, {'name': 'Apsaugos darbas', 'href': "javascript:__doPostBack('ctl00$MainArea$GroupedPOSearchTab$ProffessionGroups$ctl02$ProfGroup','')"}, {'name': 'Gamtamokslinis darbas', 'href': "javascript:__doPostBack('ctl00$MainArea$GroupedPOSearchTab$ProffessionGroups$ctl03$ProfGroup','')"}, {'name': 'Įrengimai, techninė priežiūra', 'href': "javascript:__doPostBack('ctl00$MainArea$GroupedPOSearchTab$ProffessionGroups$ctl04$ProfGroup','')"}, {'name': 'Kompiuterija / IT', 'href': "javascript:__doPostBack('ctl00$MainArea$GroupedPOSearchTab$ProffessionGroups$ctl05$ProfGroup','')"}, {'name': 'Kultūra, sportas, žiniasklaida, dizainas', 'href': "javascript:__doPostBack('ctl00$MainArea$GroupedPOSear

In [16]:
create_resp = main.send_data_to_api(endpoint="/api/categories/create", data=cats.data)

In [19]:
create_resp.data.json()

[{'id': 121,
  'name': 'Administravimas, ekonomika',
  'href': "javascript:__doPostBack('ctl00$MainArea$GroupedPOSearchTab$ProffessionGroups$ctl00$ProfGroup','')",
  'last_id': None},
 {'id': 122,
  'name': 'Amatininkai',
  'href': "javascript:__doPostBack('ctl00$MainArea$GroupedPOSearchTab$ProffessionGroups$ctl01$ProfGroup','')",
  'last_id': None},
 {'id': 123,
  'name': 'Apsaugos darbas',
  'href': "javascript:__doPostBack('ctl00$MainArea$GroupedPOSearchTab$ProffessionGroups$ctl02$ProfGroup','')",
  'last_id': None},
 {'id': 124,
  'name': 'Gamtamokslinis darbas',
  'href': "javascript:__doPostBack('ctl00$MainArea$GroupedPOSearchTab$ProffessionGroups$ctl03$ProfGroup','')",
  'last_id': None},
 {'id': 125,
  'name': 'Įrengimai, techninė priežiūra',
  'href': "javascript:__doPostBack('ctl00$MainArea$GroupedPOSearchTab$ProffessionGroups$ctl04$ProfGroup','')",
  'last_id': None},
 {'id': 126,
  'name': 'Kompiuterija / IT',
  'href': "javascript:__doPostBack('ctl00$MainArea$GroupedPOSear

In [ ]:
json.dumps(cats.data)

In [ ]:
#open category
uzt.submit_asp_form(cats[0].href)
uzt.get(url=uzt.next_url)

In [ ]:
uzt.next_url

In [ ]:
uzt.save_to_html("save00_card.html")

In [ ]:
next_page_href = get_next_page_href(uzt)

print(next_page_href)


In [ ]:
uzt.response.text

In [ ]:
#next page
uzt.submit_asp_form(next_page_href)
uzt.save_to_html("save00_card.html")

In [ ]:
# open page 2
uzt.submit_asp_form("javascript:__doPostBack('ctl00$MainArea$SearchResultsList$POGrid$ctl29$PageBtn10','')")
uzt.save_to_html("save00_card.html")

In [ ]:
need_to_get = ("__VIEWSTATE", "__VIEWSTATEGENERATOR", "__EVENTVALIDATION")
asp_inputs = uzt.asp_form_inputs
asp_inputs

In [ ]:
uzt.get(url=uzt.next_url)

In [ ]:
jobs_list = list()

In [ ]:
# поиск строк и ячеек
table = uzt.tree.css_first("#ctl00_MainArea_SearchResultsList_POGrid")
tr_list = table.css("tr:not(:nth-child(-n+2)):not(:last-child)")
for tr in tr_list:
    date1, date2, title, company, place = [item.text().strip() for item in tr.css("td")]
    jobs_list.append(Job(date1=date1, date2=date2, title=title, company=company, place=place).dict())
    
print(len(jobs_list))
print(json.dumps(jobs_list))


In [ ]:
# open job's card
uzt.submit_asp_form("javascript:__doPostBack('ctl00$MainArea$SearchResultsList$POGrid$ctl03$PositionTitleButton','')")

In [ ]:
uzt.text

In [ ]:
uzt.asp_form_inputs
